In [1]:
!pip install tensorflow tensorflow-gpu opencv-python matplotlib

     -------------------------------------- 455.9/455.9 MB 4.3 MB/s eta 0:00:00
     -------------------------------------- 455.9/455.9 MB 1.3 MB/s eta 0:00:00
     --------------------------------------- 35.6/35.6 MB 11.5 MB/s eta 0:00:00
     -------------------------------------- 124.6/124.6 kB 7.1 MB/s eta 0:00:00
  Using cached tensorflow_estimator-2.10.0-py2.py3-none-any.whl (438 kB)
     ---------------------------------------- 1.5/1.5 MB 23.8 MB/s eta 0:00:00
     ------------------------------------- 895.7/895.7 kB 14.3 MB/s eta 0:00:00
  Using cached tensorboard-2.10.1-py3-none-any.whl (5.9 MB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached keras-2.10.0-py2.py3-none-any.whl (1.7 MB)
  Using cached libclang-14.0.6-py2.py3-none-win_amd64.whl (14.2 MB)
     ---------------------------------------- 2.6/2.6 MB 18.5 MB/s eta 0:00:00
  Using cached gast-0.4.0-py3-none-any.whl (9.8 kB)
  Using cached termcolor-2.0.1-py3-none-any.whl (5.4 kB)
     --------

In [2]:
!pip list

Package                      Version
---------------------------- -----------
absl-py                      1.3.0
aiohttp                      3.8.3
aiosignal                    1.2.0
anyio                        3.5.0
argon2-cffi                  21.3.0
argon2-cffi-bindings         21.2.0
asttokens                    2.0.5
astunparse                   1.6.3
async-timeout                4.0.2
attrs                        21.4.0
Babel                        2.9.1
backcall                     0.2.0
beautifulsoup4               4.11.1
bleach                       4.1.0
blis                         0.7.9
Bottleneck                   1.3.5
brotlipy                     0.7.0
cachetools                   5.2.0
catalogue                    2.0.8
certifi                      2022.9.24
cffi                         1.15.1
charset-normalizer           2.0.4
click                        8.1.3
colorama                     0.4.5
colour                       0.1.5
confection                   0.0.3
cry

In [19]:
import tensorflow as tf
import os

In [20]:
# Avoid OOM errors by setting GPU Memory Consumption Growth
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [21]:
tf.config.list_physical_devices('GPU')

[]

In [22]:
gpus = tf.config.experimental.list_physical_devices('CPU')

In [23]:
len(gpus)

1

In [24]:
gpus_local = tf.config.experimental.list_logical_devices('GPU')

In [25]:
len(gpus_local)

0

In [28]:
pip install cv

ModuleNotFoundError: No module named 'cv2'